<a href="https://colab.research.google.com/github/khanmhmdi/Moe-llm-edge-computing/blob/main/moe_classification_switch_transformer_placment_model_newtork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os

# ------------------------------
# 1. DataLoader for MNIST
# ------------------------------
def get_data_loaders(batch_size=64):
    """
    Returns train and test DataLoader for MNIST dataset.
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, test_loader


# ------------------------------
# 2. MoE Components
# ------------------------------

class GatingNetwork(nn.Module):
    """
    A gating network that outputs a distribution over experts
    and the top-k selected expert indices.
    """
    def __init__(self, in_channels, num_experts, top_k=1):
        super(GatingNetwork, self).__init__()
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(in_channels, num_experts)
        self.top_k = top_k
        # We'll store the last top_k indices for external use (placement training)
        self.last_topk_indices = None

    def forward(self, x):
        batch_size = x.size(0)
        pooled = self.global_pool(x).view(batch_size, -1)
        logits = self.fc(pooled)  # shape: (batch_size, num_experts)

        topk_logits, topk_indices = torch.topk(logits, self.top_k, dim=1)
        topk_weights = F.softmax(topk_logits, dim=1)

        # Create a sparse weights tensor (batch_size, num_experts) with zeros everywhere
        sparse_weights = torch.zeros_like(logits)
        sparse_weights.scatter_(1, topk_indices, topk_weights)

        # Store these indices for later usage (placement)
        self.last_topk_indices = topk_indices.detach()

        return sparse_weights, topk_indices


class Expert(nn.Module):
    """
    A simple CNN expert: Conv -> BN -> ReLU.
    """
    def __init__(self, in_channels, out_channels):
        super(Expert, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))


class MoEBlock(nn.Module):
    """
    Mixture-of-Experts block:
     - Gating network to select top_k experts
     - Weighted sum of expert outputs
     - Optional load-balance calculation
    """
    def __init__(self, in_channels, out_channels, num_experts=4, top_k=1):
        super(MoEBlock, self).__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        self.experts = nn.ModuleList([Expert(in_channels, out_channels)
                                      for _ in range(num_experts)])
        self.gate = GatingNetwork(in_channels, num_experts, top_k=top_k)
        self.out_conv = nn.Conv2d(out_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        # gating
        gate_weights, topk_indices = self.gate(x)

        # Compute each expert’s output
        expert_outputs = [expert(x) for expert in self.experts]
        # Stack along a new dimension => shape: (batch, num_experts, C, H, W)
        expert_stack = torch.stack(expert_outputs, dim=1)
        # Reshape gate weights => (batch, num_experts, 1, 1, 1)
        gate_weights = gate_weights.view(-1, self.num_experts, 1, 1, 1)
        # Weighted sum
        out = (expert_stack * gate_weights).sum(dim=1)

        # A little post-processing
        out = self.relu(self.bn(self.out_conv(out)))

        # (Optional) load balance loss or usage stats
        # We'll define them as placeholders so the signature remains
        batch_size = x.size(0)
        importance = gate_weights.squeeze().sum(0)
        diversity = (gate_weights.squeeze() > 0).float().sum(0)
        mean_importance = importance / batch_size
        mean_diversity = diversity / batch_size
        load_balance_loss = (mean_importance * mean_diversity).sum()

        # Expert distribution
        flat_indices = topk_indices.view(-1)
        counts = torch.bincount(flat_indices, minlength=self.num_experts)
        proportions = counts.float() / (batch_size * self.top_k)

        return out, load_balance_loss, proportions


class MoEClassifier(nn.Module):
    """
    A simple 2-layer MoE CNN for MNIST classification.
    """
    def __init__(self, num_classes=10, num_experts=4, top_k=1):
        super(MoEClassifier, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        self.pool = nn.MaxPool2d(2)

        self.moe_block1 = MoEBlock(32, 64, num_experts, top_k)
        self.moe_block2 = MoEBlock(64, 128, num_experts, top_k)

        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        lb_loss = 0.0
        gate_dists = []

        x = self.initial(x)
        x = self.pool(x)

        x, lb1, dist1 = self.moe_block1(x)
        lb_loss += lb1
        gate_dists.append(dist1)

        x = self.pool(x)
        x, lb2, dist2 = self.moe_block2(x)
        lb_loss += lb2
        gate_dists.append(dist2)

        x = self.global_pool(x).view(x.size(0), -1)
        logits = self.fc(x)

        return logits, lb_loss, gate_dists


# ------------------------------
# 3. Training and Testing (Base Model)
# ------------------------------
def train_base_model(model, device, train_loader, optimizer, criterion, epoch, aux_coeff):
    """
    Train loop for the MoE base model.
    """
    model.train()
    total_loss = 0.0
    correct = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        logits, aux_loss, gate_dists = model(data)
        loss = criterion(logits, target) + aux_coeff * aux_loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = logits.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}]")
            print(f"  Loss: {loss.item():.6f}")

    avg_loss = total_loss / len(train_loader)
    accuracy = 100.0 * correct / len(train_loader.dataset)
    print(f"Epoch {epoch} - Base Model Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")


def tes_base_model(model, device, test_loader, criterion):
    """
    Test loop for the MoE base model.
    """
    model.eval()
    test_loss = 0.0
    correct = 0
    total_samples = 0

    # For distribution stats
    gate1_total = torch.zeros(model.moe_block1.num_experts).to(device)
    gate2_total = torch.zeros(model.moe_block2.num_experts).to(device)

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            batch_size = data.size(0)
            logits, _, gate_dists = model(data)

            loss = criterion(logits, target)
            test_loss += loss.item()
            pred = logits.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total_samples += batch_size

            # Accumulate gating distribution
            gate1_total += gate_dists[0] * batch_size
            gate2_total += gate_dists[1] * batch_size

    test_loss /= len(test_loader)
    accuracy = 100.0 * correct / total_samples
    print(f"Base Model Test - Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")

    # Print average distribution
    gate1_avg = (gate1_total / total_samples).cpu().numpy()
    gate2_avg = (gate2_total / total_samples).cpu().numpy()
    print("Block1 Dist (%):", " ".join([f"{100*d:.1f}" for d in gate1_avg]))
    print("Block2 Dist (%):", " ".join([f"{100*d:.1f}" for d in gate2_avg]))


# ------------------------------
# 4. Network Topology (Server <-> Devices)
# ------------------------------
class NetworkTopology:
    """
    Minimal example: server is ID=0, devices are 1..N.
    'comm_cost[i]' is cost from server to device i.
    """
    def __init__(self, num_devices=3, base_cost=5.0):
        self.num_devices = num_devices
        # cost[0] for server => 0
        # cost[i] for device i => base_cost + small random
        self.comm_cost = [0.0]
        for i in range(1, num_devices+1):
            cost_i = base_cost + np.random.rand() * 2.0
            self.comm_cost.append(cost_i)

    def get_cost(self, device_id):
        return self.comm_cost[device_id]


# ------------------------------
# 5. Placement Selector Model
# ------------------------------
class PlacementSelector(nn.Module):
    """
    Predicts device placements for each selected expert.
    For demonstration: uses an embedding of (layer_id, expert_id) to
    produce a distribution over device_id in [0..num_devices].
    """
    def __init__(self, num_layers, num_experts, num_devices=3, embedding_dim=16):
        super(PlacementSelector, self).__init__()
        self.num_devices = num_devices
        # total_expert_keys = num_layers * num_experts
        self.num_layers = num_layers
        self.num_experts = num_experts

        # We create a single embedding to handle (layer_id, expert_id) => index
        self.embedding = nn.Embedding(num_layers * num_experts, embedding_dim)
        # Output distribution over [0..num_devices], i.e. (num_devices+1) possibilities
        self.fc = nn.Linear(embedding_dim, num_devices+1)

    def forward(self, layer_ids, expert_ids):
        """
        :param layer_ids:   (batch, top_k)
        :param expert_ids:  (batch, top_k)
        :return: device_probs => (batch, top_k, num_devices+1)
        """
        # Convert (layer_id, expert_id) to single index
        # index = layer_id * num_experts + expert_id
        combined_key = layer_ids * self.num_experts + expert_ids
        flat_key = combined_key.view(-1)
        emb = self.embedding(flat_key)  # shape: (batch*top_k, embedding_dim)
        logits = self.fc(emb)           # shape: (batch*top_k, num_devices+1)
        device_probs = F.softmax(logits, dim=-1)
        # Reshape
        B, K = combined_key.shape
        device_probs = device_probs.view(B, K, -1)
        return device_probs


# ------------------------------
# 6. Communication-Cost–Based Loss
# ------------------------------
def compute_placement_loss(device_probs, topology):
    """
    Simple version: we pick argmax as the device assignment (non-differentiable).
    Then cost = topology.get_cost(assigned_device).
    best_cost = minimal cost among all devices.
    Loss = chosen_cost - best_cost, averaged over all samples.
    """
    batch_size, top_k, num_dev_plus_one = device_probs.size()
    chosen_device = torch.argmax(device_probs, dim=-1)  # shape: (batch_size, top_k)

    total_diff = 0.0
    # We'll do a simple loop for clarity
    for b in range(batch_size):
        for k in range(top_k):
            d_id = chosen_device[b, k].item()  # integer device ID
            chosen_cost = topology.get_cost(d_id)
            # best possible cost
            best_cost = min(topology.get_cost(d) for d in range(num_dev_plus_one))
            total_diff += (chosen_cost - best_cost)

    avg_loss = total_diff / (batch_size * top_k)
    return avg_loss


# ------------------------------
# 7. Train the Placement Selector
# ------------------------------
def train_placement_selector(base_model, placement_model, device,
                             train_loader, optimizer, topology, num_layers):
    """
    Only the placement_model is trained. The base_model is frozen.
    We'll collect top-k indices from each MoEBlock's gating.
    """
    base_model.eval()
    placement_model.train()

    total_loss = 0.0
    count = 0

    # We expect 2 MoE blocks => layer_ids: block1->0, block2->1 (for example)
    # Adjust if more layers are present.

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        _ = target.to(device)  # we don't actually need target for cost-based training

        optimizer.zero_grad()

        # Forward pass (frozen) to get top-k indices from each block
        with torch.no_grad():
            _ = base_model(data)  # This populates gating networks with .last_topk_indices

        # For block1
        topk_indices_block1 = base_model.moe_block1.gate.last_topk_indices  # shape: (batch, top_k)
        layer_ids_block1 = torch.zeros_like(topk_indices_block1, device=device)  # fill with 0

        # For block2
        topk_indices_block2 = base_model.moe_block2.gate.last_topk_indices
        layer_ids_block2 = torch.ones_like(topk_indices_block2, device=device)  # fill with 1

        # Placement model forward
        device_probs_block1 = placement_model(layer_ids_block1, topk_indices_block1)
        device_probs_block2 = placement_model(layer_ids_block2, topk_indices_block2)

        # Compute cost-based loss
        loss_block1 = compute_placement_loss(device_probs_block1, topology)
        loss_block2 = compute_placement_loss(device_probs_block2, topology)
        loss = loss_block1 + loss_block2

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        count += 1

        if batch_idx % 100 == 0:
            print(f"[{batch_idx}/{len(train_loader)}] Placement Loss: {loss.item():.4f}")

    avg_loss = total_loss / max(1, count)
    print(f"Placement Training - Avg Cost Loss: {avg_loss:.4f}")


# ------------------------------
# 8. Main: Train base model, then freeze & train placement
# ------------------------------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # Prepare data
    train_loader, test_loader = get_data_loaders(batch_size=64)

    # -------------------------
    # A) Train the base MoE model
    # -------------------------
    num_experts = 8
    top_k = 2
    base_model = MoEClassifier(num_experts=num_experts, top_k=top_k).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(base_model.parameters(), lr=0.001)
    aux_coeff = 2.0

    print("\n--- Training Base MoE Model ---")
    base_epochs = 2  # keep it small for demo
    for epoch in range(1, base_epochs+1):
        train_base_model(base_model, device, train_loader, optimizer, criterion, epoch, aux_coeff)
        tes_base_model(base_model, device, test_loader, criterion)

    # Save the trained model
    base_model_path = "moe_mnist_balanced.pth"
    torch.save(base_model.state_dict(), base_model_path)
    print(f"Base model saved to {base_model_path}.")

    # -------------------------
    # B) Load the pretrained MoE in a frozen state
    # -------------------------
    frozen_model = MoEClassifier(num_experts=num_experts, top_k=top_k).to(device)
    frozen_model.load_state_dict(torch.load(base_model_path, map_location=device))

    for param in frozen_model.parameters():
        param.requires_grad = False

    # -------------------------
    # C) Create a topology and the PlacementSelector
    # -------------------------
    num_devices = 3  # number of edge devices
    topology = NetworkTopology(num_devices=num_devices, base_cost=5.0)

    # We have 2 MoE blocks => num_layers=2 in the base model
    placement_model = PlacementSelector(num_layers=2,
                                       num_experts=num_experts,
                                       num_devices=num_devices,
                                       embedding_dim=16).to(device)

    placement_optimizer = optim.Adam(placement_model.parameters(), lr=1e-3)

    # -------------------------
    # D) Train the placement selector
    # -------------------------
    print("\n--- Training Placement Selector ---")
    placement_epochs = 3
    for epoch in range(1, placement_epochs+1):
        print(f"=== Placement Epoch {epoch} ===")
        train_placement_selector(frozen_model, placement_model, device,
                                 train_loader, placement_optimizer,
                                 topology, num_layers=2)

    # Save the trained placement model
    placement_path = "placement_selector.pth"
    torch.save(placement_model.state_dict(), placement_path)
    print(f"Placement selector saved to {placement_path}.")


if __name__ == "__main__":
    # Make sure we're not cluttering with old plots, etc.
    # You can remove or comment out these lines if desired
    if not os.path.exists("./data"):
        os.makedirs("./data")
    main()
